In [ ]:
import sys
import urllib.request
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
from scipy.spatial import distance

# Your existing functions (extract_model, draw_skeleton, process_frame, process_video) remain unchanged
def extract_model(zip_path, extract_to="models"):
    """Extracts a zip file and returns the extracted directory path."""
    extract_dir = os.path.join(os.path.dirname(zip_path), extract_to)

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    model_dir = os.path.join(extract_dir, os.path.splitext(os.path.basename(zip_path))[0])
    return model_dir

def draw_skeleton(frame, keypoints, skeleton_edges):
    """Draws skeleton on the frame using detected keypoints."""
    for (i, j) in skeleton_edges:
        pt1, pt2 = keypoints[i], keypoints[j]

        # Check if confidence is available (shape [N, 24, 3] expected)
        if keypoints.shape[-1] == 3:  
            if (pt1[2] > 0.5) and (pt2[2] > 0.5):  # Confidence threshold
                cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)  # Blue lines
        else:  
            # Draw without confidence check
            cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)

    return frame


def process_frame(frame, model, skeleton_edges):
    """Process a single frame using the Metrabs model and return pose visualization."""
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    original_height, original_width = frame.shape[:2]
    frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))  # Resize as needed
    image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

    # Run pose estimation
    pred = model.detect_poses(image_tensor, skeleton='smpl_24')
    print("Prediction output:", pred)  # Debugging output

    if 'poses2d' in pred:
        keypoints = pred['poses2d'].numpy()
        print("Keypoints shape:", keypoints.shape)

        if keypoints.shape[0] == 0:
            print("No pose detected in this frame.")
            return frame

        # Convert normalized keypoints to pixel coordinates
        for keypoint in keypoints:
            keypoint[:, 0] *= original_width / 640  # Scale x-coordinates
            keypoint[:, 1] *= original_height / 480  # Scale y-coordinates

            frame = draw_skeleton(frame, keypoint, skeleton_edges)

    return frame

def process_video(video_path, model, skeleton_edges, output_video="output_video3d-36.mp4", frame_interval=10, output_excel="2dand3dposedata.xlsx"):
    """Process video frames with pose estimation, overlay results, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_height, original_width = frame.shape[:2]
            frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
            image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

            # Run pose estimation
            pred = model.detect_poses(image_tensor, skeleton='smpl_24')

            if 'poses2d' in pred and 'poses3d' in pred:
                keypoints_2d = pred['poses2d'].numpy()
                keypoints_3d = pred['poses3d'].numpy()

                if keypoints_2d.shape[0] == 0:
                    print(f"No pose detected in frame {frame_count}")
                else:
                    # Convert normalized keypoints to pixel coordinates
                    for i in range(keypoints_2d.shape[0]):  # Iterate over detected poses
                        if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                            kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                            kpt_3d = keypoints_3d[i]

                            # Scale keypoints to original frame size
                            kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                            kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                            frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                            # Store frame-wise keypoints
                            for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                            
                                data_list.append([
                                    frame_count, 
                                    i, 
                                    j, 
                                    kpt_2d[j][0], 
                                    kpt_2d[j][1], 
                                      # Populate confidence for 2D keypoints
                                    kpt_3d[j][0], 
                                    kpt_3d[j][1], 
                                    kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                                     # Populate confidence for 3D keypoints
                                ])
                        else:
                            print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out.write(frame)  # Write frame to output video
        frame_count += 1

    cap.release()
    out.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")

def calculate_gait_metrics(df, fps, skeleton_edges, joint_names):
    """Calculate gait metrics from 2D and 3D pose data."""
    # Key joint indices for SMPL_24 (adjust based on your model documentation)
    # Example: Assuming pelvis=0, left_hip=1, left_knee=4, left_ankle=7, right_hip=2, right_knee=5, right_ankle=8
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    # Filter data for one person (assuming Person=0)
    df_person = df[df['Person'] == 0]

    # Extract key joint coordinates over time
    left_ankle_2d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    right_ankle_2d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    left_ankle_3d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    right_ankle_3d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values

    # Detect heel strikes (simplified: lowest Y/Z position of ankle)
    left_strikes = []
    right_strikes = []
    for i in range(1, len(left_ankle_3d) - 1):
        if left_ankle_3d[i][3] < left_ankle_3d[i-1][3] and left_ankle_3d[i][3] < left_ankle_3d[i+1][3]:  # Local minima in Z
            left_strikes.append(left_ankle_3d[i])
        if right_ankle_3d[i][3] < right_ankle_3d[i-1][3] and right_ankle_3d[i][3] < right_ankle_3d[i+1][3]:
            right_strikes.append(right_ankle_3d[i])

    # Convert to numpy arrays
    left_strikes = np.array(left_strikes)
    right_strikes = np.array(right_strikes)

    # 1. Stride Time (seconds between consecutive strikes of same foot)
    if len(left_strikes) > 1:
        stride_times_left = np.diff(left_strikes[:, 0]) / fps  # Frame difference to seconds
        stride_time_left = np.mean(stride_times_left)
    else:
        stride_time_left = np.nan
    if len(right_strikes) > 1:
        stride_times_right = np.diff(right_strikes[:, 0]) / fps
        stride_time_right = np.mean(stride_times_right)
    else:
        stride_time_right = np.nan

    # 2. Stride Length (distance between consecutive strikes in 3D)
    if len(left_strikes) > 1:
        stride_lengths_left = [distance.euclidean(left_strikes[i][1:4], left_strikes[i+1][1:4]) for i in range(len(left_strikes)-1)]
        stride_length_left = np.mean(stride_lengths_left)
    else:
        stride_length_left = np.nan
    if len(right_strikes) > 1:
        stride_lengths_right = [distance.euclidean(right_strikes[i][1:4], right_strikes[i+1][1:4]) for i in range(len(right_strikes)-1)]
        stride_length_right = np.mean(stride_lengths_right)
    else:
        stride_length_right = np.nan

    # 3. Cadence (steps per minute)
    total_steps = len(left_strikes) + len(right_strikes)
    total_time = (df['Frame'].max() - df['Frame'].min()) / fps / 60  # Total time in minutes
    cadence = total_steps / total_time if total_time > 0 else np.nan

    # 4. Gait Speed (meters per second, assuming stride length in meters)
    gait_speed_left = stride_length_left / stride_time_left if stride_time_left > 0 else np.nan
    gait_speed_right = stride_length_right / stride_time_right if stride_time_right > 0 else np.nan

    # 5. Knee and Hip Angles (frame-by-frame for 2D and 3D)
    angles_2d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    angles_3d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}

    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        # Extract coordinates for joints
        def get_coords(joint, dim='2D'):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            if dim == '2D':
                return row[['X_2D', 'Y_2D']].values[0] if not row.empty else np.array([np.nan, np.nan])
            else:
                return row[['X_3D', 'Y_3D', 'Z_3D']].values[0] if not row.empty else np.array([np.nan, np.nan, np.nan])

        # 2D vectors
        left_hip_2d = get_coords('left_hip', '2D')
        left_knee_2d = get_coords('left_knee', '2D')
        left_ankle_2d = get_coords('left_ankle', '2D')
        right_hip_2d = get_coords('right_hip', '2D')
        right_knee_2d = get_coords('right_knee', '2D')
        right_ankle_2d = get_coords('right_ankle', '2D')
        spine_2d = get_coords('spine1', '2D')

        # 3D vectors
        left_hip_3d = get_coords('left_hip', '3D')
        left_knee_3d = get_coords('left_knee', '3D')
        left_ankle_3d = get_coords('left_ankle', '3D')
        right_hip_3d = get_coords('right_hip', '3D')
        right_knee_3d = get_coords('right_knee', '3D')
        right_ankle_3d = get_coords('right_ankle', '3D')
        spine_3d = get_coords('spine1', '3D')

        # Calculate angles (degrees)
        def angle_between(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        # Knee angles
        angles_2d['left_knee'].append(angle_between(left_hip_2d - left_knee_2d, left_ankle_2d - left_knee_2d))
        angles_2d['right_knee'].append(angle_between(right_hip_2d - right_knee_2d, right_ankle_2d - right_knee_2d))
        angles_3d['left_knee'].append(angle_between(left_hip_3d - left_knee_3d, left_ankle_3d - left_knee_3d))
        angles_3d['right_knee'].append(angle_between(right_hip_3d - right_knee_3d, right_ankle_3d - right_knee_3d))

        # Hip angles
        angles_2d['left_hip'].append(angle_between(spine_2d - left_hip_2d, left_knee_2d - left_hip_2d))
        angles_2d['right_hip'].append(angle_between(spine_2d - right_hip_2d, right_knee_2d - right_hip_2d))
        angles_3d['left_hip'].append(angle_between(spine_3d - left_hip_3d, left_knee_3d - left_hip_3d))
        angles_3d['right_hip'].append(angle_between(spine_3d - right_hip_3d, right_knee_3d - right_hip_3d))

    # Average angles
    avg_angles_2d = {k: np.nanmean(v) for k, v in angles_2d.items()}
    avg_angles_3d = {k: np.nanmean(v) for k, v in angles_3d.items()}

    # Compile results
    metrics = {
        'Stride Time Left (s)': stride_time_left,
        'Stride Time Right (s)': stride_time_right,
        'Stride Length Left (m)': stride_length_left,
        'Stride Length Right (m)': stride_length_right,
        'Cadence (steps/min)': cadence,
        'Gait Speed Left (m/s)': gait_speed_left,
        'Gait Speed Right (m/s)': gait_speed_right,
        'Avg Left Knee Angle 2D (deg)': avg_angles_2d['left_knee'],
        'Avg Right Knee Angle 2D (deg)': avg_angles_2d['right_knee'],
        'Avg Left Hip Angle 2D (deg)': avg_angles_2d['left_hip'],
        'Avg Right Hip Angle 2D (deg)': avg_angles_2d['right_hip'],
        'Avg Left Knee Angle 3D (deg)': avg_angles_3d['left_knee'],
        'Avg Right Knee Angle 3D (deg)': avg_angles_3d['right_knee'],
        'Avg Left Hip Angle 3D (deg)': avg_angles_3d['left_hip'],
        'Avg Right Hip Angle 3D (deg)': avg_angles_3d['right_hip']
    }

    return metrics

def main():
    # Paths
    zip_path = "C:\\Users\\akhileshsing2024\\Downloads\\metrabs_mob3l_y4t.zip"
    video_path = "C:\\Users\\akhileshsing2024\\Downloads\\video.mp4"
    output_excel = "2dand3dposedata.xlsx"

    # Extract and load the model
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")

    # Load skeleton information
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()

    # Process video
    process_video(video_path, model, joint_edges, output_excel=output_excel)

    # Load the generated Excel file
    df = pd.read_excel(output_excel)
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)

    # Calculate gait metrics
    metrics = calculate_gait_metrics(df, fps, joint_edges, joint_names)

    # Print and save results
    print("\nGait Metrics:")
    for key, value in metrics.items():
        print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    metrics_df = pd.DataFrame([metrics])
    metrics_df.to_excel("gait_metrics.xlsx", index=False)
    print("Gait metrics saved to 'gait_metrics.xlsx'")

if __name__ == '__main__':
    main()

Model loaded successfully!
Processed video saved as output_video3d-36.mp4
Pose data saved as 2dand3dposedata.xlsx

Gait Metrics:
Stride Time Left (s): 0.97
Stride Time Right (s): 1.08
Stride Length Left (m): 1597.56
Stride Length Right (m): 1089.24
Cadence (steps/min): 79.92
Gait Speed Left (m/s): 1641.56
Gait Speed Right (m/s): 1004.44
Avg Left Knee Angle 2D (deg): 172.90
Avg Right Knee Angle 2D (deg): 160.95
Avg Left Hip Angle 2D (deg): 170.35
Avg Right Hip Angle 2D (deg): 160.23
Avg Left Knee Angle 3D (deg): 162.53
Avg Right Knee Angle 3D (deg): 161.61
Avg Left Hip Angle 3D (deg): 168.36
Avg Right Hip Angle 3D (deg): 163.25
Gait metrics saved to 'gait_metrics.xlsx'


In [2]:
pwd

'c:\\Users\\akhileshsing2024\\AppData\\Local\\Programs\\Microsoft VS Code'

In [1]:
import sys
import urllib.request
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
from scipy.spatial import distance

def extract_model(zip_path, extract_to="models"):
    """Extracts a zip file and returns the extracted directory path."""
    extract_dir = os.path.join(os.path.dirname(zip_path), extract_to)

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    model_dir = os.path.join(extract_dir, os.path.splitext(os.path.basename(zip_path))[0])
    return model_dir

def draw_skeleton(frame, keypoints, skeleton_edges):
    """Draws skeleton on the frame using detected keypoints."""
    for (i, j) in skeleton_edges:
        pt1, pt2 = keypoints[i], keypoints[j]

        # Check if confidence is available (shape [N, 24, 3] expected)
        if keypoints.shape[-1] == 3:  
            if (pt1[2] > 0.5) and (pt2[2] > 0.5):  # Confidence threshold
                cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)  # Blue lines
        else:  
            # Draw without confidence check
            cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)

    return frame


# def process_frame(frame, model, skeleton_edges):
#     """Process a single frame using the Metrabs model and return pose visualization."""
#     frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#     original_height, original_width = frame.shape[:2]
#     frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))  # Resize as needed
#     image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

#     # Run pose estimation
#     pred = model.detect_poses(image_tensor, skeleton='smpl_24')
#     print("Prediction output:", pred)  # Debugging output

#     if 'poses2d' in pred:
#         keypoints = pred['poses2d'].numpy()
#         print("Keypoints shape:", keypoints.shape)

#         if keypoints.shape[0] == 0:
#             print("No pose detected in this frame.")
#             return frame

#         # Convert normalized keypoints to pixel coordinates
#         for keypoint in keypoints:
#             keypoint[:, 0] *= original_width / 640  # Scale x-coordinates
#             keypoint[:, 1] *= original_height / 480  # Scale y-coordinates

#             frame = draw_skeleton(frame, keypoint, skeleton_edges)

#     return frame


def process_video(video_path, model, skeleton_edges, output_video="output_video3d-36.mp4", frame_interval=10, output_excel="2dand3dposedata.xlsx"):
    """Process video frames with pose estimation, overlay results, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_height, original_width = frame.shape[:2]
            frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
            image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

            # Run pose estimation
            pred = model.detect_poses(image_tensor, skeleton='smpl_24')

            if 'poses2d' in pred and 'poses3d' in pred:
                keypoints_2d = pred['poses2d'].numpy()
                keypoints_3d = pred['poses3d'].numpy()

                if keypoints_2d.shape[0] == 0:
                    print(f"No pose detected in frame {frame_count}")
                else:
                    # Convert normalized keypoints to pixel coordinates
                    for i in range(keypoints_2d.shape[0]):  # Iterate over detected poses
                        if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                            kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                            kpt_3d = keypoints_3d[i]

                            # Scale keypoints to original frame size
                            kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                            kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                            frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                            # Store frame-wise keypoints
                            for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                            
                                data_list.append([
                                    frame_count, 
                                    i, 
                                    j, 
                                    kpt_2d[j][0], 
                                    kpt_2d[j][1], 
                                      # Populate confidence for 2D keypoints
                                    kpt_3d[j][0], 
                                    kpt_3d[j][1], 
                                    kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                                     # Populate confidence for 3D keypoints
                                ])
                                
                        else:
                            print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out.write(frame)  # Write frame to output video
        frame_count += 1

    cap.release()
    out.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")


def calculate_gait_metrics_2d(df, fps, skeleton_edges, joint_names):
    """Calculate gait metrics using only 2D coordinates."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == 0]
    left_ankle_2d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    right_ankle_2d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_2D', 'Y_2D']].values

    # Detect heel strikes in 2D (lowest Y_2D position)
    left_strikes_2d = []
    right_strikes_2d = []
    for i in range(1, len(left_ankle_2d) - 1):
        if left_ankle_2d[i][2] < left_ankle_2d[i-1][2] and left_ankle_2d[i][2] < left_ankle_2d[i+1][2]:  # Local minima in Y_2D
            left_strikes_2d.append(left_ankle_2d[i])
        if right_ankle_2d[i][2] < right_ankle_2d[i-1][2] and right_ankle_2d[i][2] < right_ankle_2d[i+1][2]:
            right_strikes_2d.append(right_ankle_2d[i])

    left_strikes_2d = np.array(left_strikes_2d)
    right_strikes_2d = np.array(right_strikes_2d)

    # 1. Stride Time (2D)
    stride_time_left_2d = np.mean(np.diff(left_strikes_2d[:, 0]) / fps) if len(left_strikes_2d) > 1 else np.nan
    stride_time_right_2d = np.mean(np.diff(right_strikes_2d[:, 0]) / fps) if len(right_strikes_2d) > 1 else np.nan

    # 2. Stride Length (2D, in pixels)
    stride_length_left_2d = np.mean([distance.euclidean(left_strikes_2d[i][1:3], left_strikes_2d[i+1][1:3]) 
                                    for i in range(len(left_strikes_2d)-1)]) if len(left_strikes_2d) > 1 else np.nan
    stride_length_right_2d = np.mean([distance.euclidean(right_strikes_2d[i][1:3], right_strikes_2d[i+1][1:3]) 
                                     for i in range(len(right_strikes_2d)-1)]) if len(right_strikes_2d) > 1 else np.nan

    # 3. Cadence (2D)
    total_steps_2d = len(left_strikes_2d) + len(right_strikes_2d)
    total_time_2d = (df['Frame'].max() - df['Frame'].min()) / fps / 60
    cadence_2d = total_steps_2d / total_time_2d if total_time_2d > 0 else np.nan

    # 4. Gait Speed (2D, pixels per second)
    gait_speed_left_2d = stride_length_left_2d / stride_time_left_2d if stride_time_left_2d > 0 else np.nan
    gait_speed_right_2d = stride_length_right_2d / stride_time_right_2d if stride_time_right_2d > 0 else np.nan

    # 5. Knee and Hip Angles (2D)
    angles_2d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_2d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_2D', 'Y_2D']].values[0] if not row.empty else np.array([np.nan, np.nan])

        left_hip_2d = get_coords_2d('left_hip')
        left_knee_2d = get_coords_2d('left_knee')
        left_ankle_2d = get_coords_2d('left_ankle')
        right_hip_2d = get_coords_2d('right_hip')
        right_knee_2d = get_coords_2d('right_knee')
        right_ankle_2d = get_coords_2d('right_ankle')
        spine_2d = get_coords_2d('spine1')

        def angle_between_2d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_2d['left_knee'].append(angle_between_2d(left_hip_2d - left_knee_2d, left_ankle_2d - left_knee_2d))
        angles_2d['right_knee'].append(angle_between_2d(right_hip_2d - right_knee_2d, right_ankle_2d - right_knee_2d))
        angles_2d['left_hip'].append(angle_between_2d(spine_2d - left_hip_2d, left_knee_2d - left_hip_2d))
        angles_2d['right_hip'].append(angle_between_2d(spine_2d - right_hip_2d, right_knee_2d - right_hip_2d))

    avg_angles_2d = {k: np.nanmean(v) for k, v in angles_2d.items()}

    return {
        'Stride Time Left (s)': stride_time_left_2d,
        'Stride Time Right (s)': stride_time_right_2d,
        'Stride Length Left (pixels)': stride_length_left_2d,
        'Stride Length Right (pixels)': stride_length_right_2d,
        'Cadence (steps/min)': cadence_2d,
        'Gait Speed Left (pixels/s)': gait_speed_left_2d,
        'Gait Speed Right (pixels/s)': gait_speed_right_2d,
        'Avg Left Knee Angle (deg)': avg_angles_2d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_2d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_2d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_2d['right_hip']
    }

def calculate_gait_metrics_3d(df, fps, skeleton_edges, joint_names):
    """Calculate gait metrics using only 3D coordinates."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == 0]
    left_ankle_3d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    right_ankle_3d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values

    # Detect heel strikes in 3D (lowest Z_3D position)
    left_strikes_3d = []
    right_strikes_3d = []
    for i in range(1, len(left_ankle_3d) - 1):
        if left_ankle_3d[i][3] < left_ankle_3d[i-1][3] and left_ankle_3d[i][3] < left_ankle_3d[i+1][3]:  # Local minima in Z_3D
            left_strikes_3d.append(left_ankle_3d[i])
        if right_ankle_3d[i][3] < right_ankle_3d[i-1][3] and right_ankle_3d[i][3] < right_ankle_3d[i+1][3]:
            right_strikes_3d.append(right_ankle_3d[i])

    left_strikes_3d = np.array(left_strikes_3d)
    right_strikes_3d = np.array(right_strikes_3d)

    # 1. Stride Time (3D)
    stride_time_left_3d = np.mean(np.diff(left_strikes_3d[:, 0]) / fps) if len(left_strikes_3d) > 1 else np.nan
    stride_time_right_3d = np.mean(np.diff(right_strikes_3d[:, 0]) / fps) if len(right_strikes_3d) > 1 else np.nan

    # 2. Stride Length (3D, in model units, assumed meters)
    stride_length_left_3d = np.mean([distance.euclidean(left_strikes_3d[i][1:4], left_strikes_3d[i+1][1:4]) 
                                    for i in range(len(left_strikes_3d)-1)]) if len(left_strikes_3d) > 1 else np.nan
    stride_length_right_3d = np.mean([distance.euclidean(right_strikes_3d[i][1:4], right_strikes_3d[i+1][1:4]) 
                                     for i in range(len(right_strikes_3d)-1)]) if len(right_strikes_3d) > 1 else np.nan

    # 3. Cadence (3D)
    total_steps_3d = len(left_strikes_3d) + len(right_strikes_3d)
    total_time_3d = (df['Frame'].max() - df['Frame'].min()) / fps / 60
    cadence_3d = total_steps_3d / total_time_3d if total_time_3d > 0 else np.nan

    # 4. Gait Speed (3D, assumed meters per second)
    gait_speed_left_3d = stride_length_left_3d / stride_time_left_3d if stride_time_left_3d > 0 else np.nan
    gait_speed_right_3d = stride_length_right_3d / stride_time_right_3d if stride_time_right_3d > 0 else np.nan

    # 5. Knee and Hip Angles (3D)
    angles_3d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_3d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_3D', 'Y_3D', 'Z_3D']].values[0] if not row.empty else np.array([np.nan, np.nan, np.nan])

        left_hip_3d = get_coords_3d('left_hip')
        left_knee_3d = get_coords_3d('left_knee')
        left_ankle_3d = get_coords_3d('left_ankle')
        right_hip_3d = get_coords_3d('right_hip')
        right_knee_3d = get_coords_3d('right_knee')
        right_ankle_3d = get_coords_3d('right_ankle')
        spine_3d = get_coords_3d('spine1')

        def angle_between_3d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_3d['left_knee'].append(angle_between_3d(left_hip_3d - left_knee_3d, left_ankle_3d - left_knee_3d))
        angles_3d['right_knee'].append(angle_between_3d(right_hip_3d - right_knee_3d, right_ankle_3d - right_knee_3d))
        angles_3d['left_hip'].append(angle_between_3d(spine_3d - left_hip_3d, left_knee_3d - left_hip_3d))
        angles_3d['right_hip'].append(angle_between_3d(spine_3d - right_hip_3d, right_knee_3d - right_hip_3d))

    avg_angles_3d = {k: np.nanmean(v) for k, v in angles_3d.items()}

    return {
        'Stride Time Left (s)': stride_time_left_3d,
        'Stride Time Right (s)': stride_time_right_3d,
        'Stride Length Left (m)': stride_length_left_3d,
        'Stride Length Right (m)': stride_length_right_3d,
        'Cadence (steps/min)': cadence_3d,
        'Gait Speed Left (m/s)': gait_speed_left_3d,
        'Gait Speed Right (m/s)': gait_speed_right_3d,
        'Avg Left Knee Angle (deg)': avg_angles_3d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_3d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_3d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_3d['right_hip']
    }

def main():
    # Paths
    zip_path = "C:\\Users\\akhileshsing2024\\Downloads\\metrabs_mob3l_y4t.zip"
    video_path = "C:\\Users\\akhileshsing2024\\Downloads\\video.mp4"
    output_excel = "2dand3dposedata.xlsx"

    # Extract and load the model
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")

    # Load skeleton information
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()

    # Process video
    process_video(video_path, model, joint_edges, output_excel=output_excel)

    # Load the generated Excel file
    df = pd.read_excel(output_excel)
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)

    # Calculate gait metrics separately
    metrics_2d = calculate_gait_metrics_2d(df, fps, joint_edges, joint_names)
    metrics_3d = calculate_gait_metrics_3d(df, fps, joint_edges, joint_names)

    # Print results
    print("\n2D Gait Metrics:")
    for key, value in metrics_2d.items():
        print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    print("\n3D Gait Metrics:")
    for key, value in metrics_3d.items():
        print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    # Save to Excel
    metrics_df_2d = pd.DataFrame([metrics_2d])
    metrics_df_3d = pd.DataFrame([metrics_3d])
    with pd.ExcelWriter("gait_metrics_separate.xlsx") as writer:
        metrics_df_2d.to_excel(writer, sheet_name='2D Metrics', index=False)
        metrics_df_3d.to_excel(writer, sheet_name='3D Metrics', index=False)
    print("Gait metrics saved to 'gait_metrics_separate.xlsx'")

if __name__ == '__main__':
    main()

Model loaded successfully!
Processed video saved as output_video3d-36.mp4
Pose data saved as 2dand3dposedata.xlsx

2D Gait Metrics:
Stride Time Left (s): 1.25
Stride Time Right (s): 1.25
Stride Length Left (pixels): 479.35
Stride Length Right (pixels): 351.42
Cadence (steps/min): 95.90
Gait Speed Left (pixels/s): 383.10
Gait Speed Right (pixels/s): 280.85
Avg Left Knee Angle (deg): 172.90
Avg Right Knee Angle (deg): 160.95
Avg Left Hip Angle (deg): 170.35
Avg Right Hip Angle (deg): 160.23

3D Gait Metrics:
Stride Time Left (s): 0.97
Stride Time Right (s): 1.08
Stride Length Left (m): 1597.56
Stride Length Right (m): 1089.24
Cadence (steps/min): 79.92
Gait Speed Left (m/s): 1641.56
Gait Speed Right (m/s): 1004.44
Avg Left Knee Angle (deg): 162.53
Avg Right Knee Angle (deg): 161.61
Avg Left Hip Angle (deg): 168.36
Avg Right Hip Angle (deg): 163.25
Gait metrics saved to 'gait_metrics_separate.xlsx'


In [2]:
pwd

'c:\\Users\\akhileshsing2024\\Downloads'

## Working code to calculate the 2d and 3d pose estimation and gait analysis


In [ ]:
import sys
import urllib.request
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
from scipy.spatial import distance

def extract_model(zip_path, extract_to="models"):
    """Extracts a zip file and returns the extracted directory path."""
    extract_dir = os.path.join(os.path.dirname(zip_path), extract_to)

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    model_dir = os.path.join(extract_dir, os.path.splitext(os.path.basename(zip_path))[0])
    return model_dir

def draw_skeleton(frame, keypoints, skeleton_edges):
    """Draws skeleton on the frame using detected keypoints."""
    for (i, j) in skeleton_edges:
        pt1, pt2 = keypoints[i], keypoints[j]

        # Check if confidence is available (shape [N, 24, 3] expected)
        if keypoints.shape[-1] == 3:  
            if (pt1[2] > 0.5) and (pt2[2] > 0.5):  # Confidence threshold
                cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)  # Blue lines
        else:  
            # Draw without confidence check
            cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)

    return frame

def process_video(video_path, model, skeleton_edges, output_video="output_video3d-36.mp4", frame_interval=10, output_excel="2dand3dposedata.xlsx"):
    """Process video frames with pose estimation, overlay results, and save coordinates to an Excel file."""
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_height, original_width = frame.shape[:2]
            frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
            image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

            # Run pose estimation
            pred = model.detect_poses(image_tensor, skeleton='smpl_24')

            if 'poses2d' in pred and 'poses3d' in pred:
                keypoints_2d = pred['poses2d'].numpy()
                keypoints_3d = pred['poses3d'].numpy()

                if keypoints_2d.shape[0] == 0:
                    print(f"No pose detected in frame {frame_count}")
                else:
                    # Convert normalized keypoints to pixel coordinates
                    for i in range(keypoints_2d.shape[0]):  # Iterate over detected poses
                        if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                            kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                            kpt_3d = keypoints_3d[i]

                            # Scale keypoints to original frame size
                            kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                            kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                            frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                            # Store frame-wise keypoints
                            for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                            
                                data_list.append([
                                    frame_count, 
                                    i, 
                                    j, 
                                    kpt_2d[j][0], 
                                    kpt_2d[j][1], 
                                      # Populate confidence for 2D keypoints
                                    kpt_3d[j][0], 
                                    kpt_3d[j][1], 
                                    kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                                     # Populate confidence for 3D keypoints
                                ])
                                
                        else:
                            print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out.write(frame)  # Write frame to output video
        frame_count += 1

    cap.release()
    out.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")
def calculate_gait_metrics_2d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 2D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_2d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    right_ankle_2d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_2D', 'Y_2D']].values

    # Detect heel strikes in 2D (lowest Y_2D position)
    left_strikes_2d = []
    right_strikes_2d = []
    for i in range(1, len(left_ankle_2d) - 1):
        if left_ankle_2d[i][2] < left_ankle_2d[i-1][2] and left_ankle_2d[i][2] < left_ankle_2d[i+1][2]:
            left_strikes_2d.append(left_ankle_2d[i])
        if right_ankle_2d[i][2] < right_ankle_2d[i-1][2] and right_ankle_2d[i][2] < right_ankle_2d[i+1][2]:
            right_strikes_2d.append(right_ankle_2d[i])

    left_strikes_2d = np.array(left_strikes_2d)
    right_strikes_2d = np.array(right_strikes_2d)

    # 1. Stride Time (2D)
    stride_time_left_2d = np.mean(np.diff(left_strikes_2d[:, 0]) / fps) if len(left_strikes_2d) > 1 else np.nan
    stride_time_right_2d = np.mean(np.diff(right_strikes_2d[:, 0]) / fps) if len(right_strikes_2d) > 1 else np.nan

    # 2. Stride Length (2D, in pixels)
    stride_length_left_2d = np.mean([distance.euclidean(left_strikes_2d[i][1:3], left_strikes_2d[i+1][1:3]) 
                                    for i in range(len(left_strikes_2d)-1)]) if len(left_strikes_2d) > 1 else np.nan
    stride_length_right_2d = np.mean([distance.euclidean(right_strikes_2d[i][1:3], right_strikes_2d[i+1][1:3]) 
                                     for i in range(len(right_strikes_2d)-1)]) if len(right_strikes_2d) > 1 else np.nan

    # 3. Cadence (2D)
    total_steps_2d = len(left_strikes_2d) + len(right_strikes_2d)
    total_time_2d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_2d = total_steps_2d / total_time_2d if total_time_2d > 0 else np.nan

    # 4. Gait Speed (2D, pixels per second)
    gait_speed_left_2d = stride_length_left_2d / stride_time_left_2d if stride_time_left_2d > 0 else np.nan
    gait_speed_right_2d = stride_length_right_2d / stride_time_right_2d if stride_time_right_2d > 0 else np.nan

    # 5. Knee and Hip Angles (2D)
    angles_2d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_2d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_2D', 'Y_2D']].values[0] if not row.empty else np.array([np.nan, np.nan])

        left_hip_2d = get_coords_2d('left_hip')
        left_knee_2d = get_coords_2d('left_knee')
        left_ankle_2d = get_coords_2d('left_ankle')
        right_hip_2d = get_coords_2d('right_hip')
        right_knee_2d = get_coords_2d('right_knee')
        right_ankle_2d = get_coords_2d('right_ankle')
        spine_2d = get_coords_2d('spine1')

        def angle_between_2d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_2d['left_knee'].append(angle_between_2d(left_hip_2d - left_knee_2d, left_ankle_2d - left_knee_2d))
        angles_2d['right_knee'].append(angle_between_2d(right_hip_2d - right_knee_2d, right_ankle_2d - right_knee_2d))
        angles_2d['left_hip'].append(angle_between_2d(spine_2d - left_hip_2d, left_knee_2d - left_hip_2d))
        angles_2d['right_hip'].append(angle_between_2d(spine_2d - right_hip_2d, right_knee_2d - right_hip_2d))

    avg_angles_2d = {k: np.nanmean(v) for k, v in angles_2d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_2d,
        'Stride Time Right (s)': stride_time_right_2d,
        'Stride Length Left (pixels)': stride_length_left_2d,
        'Stride Length Right (pixels)': stride_length_right_2d,
        'Cadence (steps/min)': cadence_2d,
        'Gait Speed Left (pixels/s)': gait_speed_left_2d,
        'Gait Speed Right (pixels/s)': gait_speed_right_2d,
        'Avg Left Knee Angle (deg)': avg_angles_2d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_2d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_2d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_2d['right_hip']
    }

def calculate_gait_metrics_3d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 3D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_3d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    right_ankle_3d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values

    # Detect heel strikes in 3D (lowest Z_3D position)
    left_strikes_3d = []
    right_strikes_3d = []
    for i in range(1, len(left_ankle_3d) - 1):
        if left_ankle_3d[i][3] < left_ankle_3d[i-1][3] and left_ankle_3d[i][3] < left_ankle_3d[i+1][3]:
            left_strikes_3d.append(left_ankle_3d[i])
        if right_ankle_3d[i][3] < right_ankle_3d[i-1][3] and right_ankle_3d[i][3] < right_ankle_3d[i+1][3]:
            right_strikes_3d.append(right_ankle_3d[i])

    left_strikes_3d = np.array(left_strikes_3d)
    right_strikes_3d = np.array(right_strikes_3d)

    # 1. Stride Time (3D)
    stride_time_left_3d = np.mean(np.diff(left_strikes_3d[:, 0]) / fps) if len(left_strikes_3d) > 1 else np.nan
    stride_time_right_3d = np.mean(np.diff(right_strikes_3d[:, 0]) / fps) if len(right_strikes_3d) > 1 else np.nan

    # 2. Stride Length (3D, assumed meters)
    stride_length_left_3d = np.mean([distance.euclidean(left_strikes_3d[i][1:4], left_strikes_3d[i+1][1:4]) 
                                    for i in range(len(left_strikes_3d)-1)]) if len(left_strikes_3d) > 1 else np.nan
    stride_length_right_3d = np.mean([distance.euclidean(right_strikes_3d[i][1:4], right_strikes_3d[i+1][1:4]) 
                                     for i in range(len(right_strikes_3d)-1)]) if len(right_strikes_3d) > 1 else np.nan

    # 3. Cadence (3D)
    total_steps_3d = len(left_strikes_3d) + len(right_strikes_3d)
    total_time_3d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_3d = total_steps_3d / total_time_3d if total_time_3d > 0 else np.nan

    # 4. Gait Speed (3D, assumed meters per second)
    gait_speed_left_3d = stride_length_left_3d / stride_time_left_3d if stride_time_left_3d > 0 else np.nan
    gait_speed_right_3d = stride_length_right_3d / stride_time_right_3d if stride_time_right_3d > 0 else np.nan

    # 5. Knee and Hip Angles (3D)
    angles_3d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_3d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_3D', 'Y_3D', 'Z_3D']].values[0] if not row.empty else np.array([np.nan, np.nan, np.nan])

        left_hip_3d = get_coords_3d('left_hip')
        left_knee_3d = get_coords_3d('left_knee')
        left_ankle_3d = get_coords_3d('left_ankle')
        right_hip_3d = get_coords_3d('right_hip')
        right_knee_3d = get_coords_3d('right_knee')
        right_ankle_3d = get_coords_3d('right_ankle')
        spine_3d = get_coords_3d('spine1')

        def angle_between_3d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_3d['left_knee'].append(angle_between_3d(left_hip_3d - left_knee_3d, left_ankle_3d - left_knee_3d))
        angles_3d['right_knee'].append(angle_between_3d(right_hip_3d - right_knee_3d, right_ankle_3d - right_knee_3d))
        angles_3d['left_hip'].append(angle_between_3d(spine_3d - left_hip_3d, left_knee_3d - left_hip_3d))
        angles_3d['right_hip'].append(angle_between_3d(spine_3d - right_hip_3d, right_knee_3d - right_hip_3d))

    avg_angles_3d = {k: np.nanmean(v) for k, v in angles_3d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_3d,
        'Stride Time Right (s)': stride_time_right_3d,
        'Stride Length Left (m)': stride_length_left_3d,
        'Stride Length Right (m)': stride_length_right_3d,
        'Cadence (steps/min)': cadence_3d,
        'Gait Speed Left (m/s)': gait_speed_left_3d,
        'Gait Speed Right (m/s)': gait_speed_right_3d,
        'Avg Left Knee Angle (deg)': avg_angles_3d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_3d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_3d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_3d['right_hip']
    }

def main():
    # Paths
    zip_path = "C:\\Users\\akhileshsing2024\\Downloads\\metrabs_mob3l_y4t.zip"
    video_path = "C:\\Users\\akhileshsing2024\\Downloads\\video.mp4"
    output_excel = "2dand3dposedata.xlsx"

    # Extract and load the model
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")

    # Load skeleton information
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()

    # Process video
    process_video(video_path, model, joint_edges, output_excel=output_excel)

    # Load the generated Excel file
    df = pd.read_excel(output_excel)
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)

    # Calculate gait metrics for each person (0 to 6)
    metrics_2d_all = []
    metrics_3d_all = []
    for person_id in range(7):  # Assuming 7 persons (0 to 6)
        print(f"\nProcessing Person {person_id}...")
        
        # 2D Metrics
        metrics_2d = calculate_gait_metrics_2d(df, person_id, fps, joint_edges, joint_names)
        if metrics_2d:
            metrics_2d_all.append(metrics_2d)
            print(f"2D Metrics for Person {person_id}:")
            for key, value in metrics_2d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

        # 3D Metrics
        metrics_3d = calculate_gait_metrics_3d(df, person_id, fps, joint_edges, joint_names)
        if metrics_3d:
            metrics_3d_all.append(metrics_3d)
            print(f"3D Metrics for Person {person_id}:")
            for key, value in metrics_3d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    # Save to Excel with separate sheets for 2D and 3D
    metrics_df_2d = pd.DataFrame(metrics_2d_all)
    metrics_df_3d = pd.DataFrame(metrics_3d_all)
    with pd.ExcelWriter("gait_metrics_per_person.xlsx") as writer:
        metrics_df_2d.to_excel(writer, sheet_name='2D Metrics', index=False)
        metrics_df_3d.to_excel(writer, sheet_name='3D Metrics', index=False)
    print("\nGait metrics for all persons saved to 'gait_metrics_per_person.xlsx'")

if __name__ == '__main__':
    main()

Model loaded successfully!
Processed video saved as output_video3d-36.mp4
Pose data saved as 2dand3dposedata.xlsx

Processing Person 0...
2D Metrics for Person 0:
Stride Time Left (s): 1.25
Stride Time Right (s): 1.25
Stride Length Left (pixels): 479.35
Stride Length Right (pixels): 351.42
Cadence (steps/min): 95.90
Gait Speed Left (pixels/s): 383.10
Gait Speed Right (pixels/s): 280.85
Avg Left Knee Angle (deg): 172.90
Avg Right Knee Angle (deg): 160.95
Avg Left Hip Angle (deg): 170.35
Avg Right Hip Angle (deg): 160.23
3D Metrics for Person 0:
Stride Time Left (s): 0.97
Stride Time Right (s): 1.08
Stride Length Left (m): 1597.56
Stride Length Right (m): 1089.24
Cadence (steps/min): 79.92
Gait Speed Left (m/s): 1641.56
Gait Speed Right (m/s): 1004.44
Avg Left Knee Angle (deg): 162.53
Avg Right Knee Angle (deg): 161.61
Avg Left Hip Angle (deg): 168.36
Avg Right Hip Angle (deg): 163.25

Processing Person 1...
2D Metrics for Person 1:
Stride Time Left (s): 1.36
Stride Time Right (s): 1.56


In [9]:
import sys
import urllib.request
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
from scipy.spatial import distance

def extract_model(zip_path, extract_to="models"):
    """Extracts a zip file and returns the extracted directory path."""
    extract_dir = os.path.join(os.path.dirname(zip_path), extract_to)

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    model_dir = os.path.join(extract_dir, os.path.splitext(os.path.basename(zip_path))[0])
    return model_dir

def draw_skeleton(frame, keypoints, skeleton_edges):
    """Draws skeleton on the frame using detected keypoints."""
    for (i, j) in skeleton_edges:
        pt1, pt2 = keypoints[i], keypoints[j]

        # Check if confidence is available (shape [N, 24, 3] expected)
        if keypoints.shape[-1] == 3:  
            if (pt1[2] > 0.5) and (pt2[2] > 0.5):  # Confidence threshold
                cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)  # Blue lines
        else:  
            # Draw without confidence check
            cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)

    return frame

# def process_video(video_path, model, skeleton_edges, output_video="output_video3d-36.mp4", frame_interval=10, output_excel="2dand3dposedata.xlsx"):
#     """Process video frames with pose estimation, overlay results, and save coordinates to an Excel file."""
    
#     cap = cv2.VideoCapture(video_path)
#     frame_width = int(cap.get(3))
#     frame_height = int(cap.get(4))
#     fps = int(cap.get(cv2.CAP_PROP_FPS))

#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

#     frame_count = 0
#     data_list = []  # Store pose data

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         if frame_count % frame_interval == 0:
#             frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             original_height, original_width = frame.shape[:2]
#             frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
#             image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

#             # Run pose estimation
#             pred = model.detect_poses(image_tensor, skeleton='smpl_24')

#             if 'poses2d' in pred and 'poses3d' in pred:
#                 keypoints_2d = pred['poses2d'].numpy()
#                 keypoints_3d = pred['poses3d'].numpy()

#                 if keypoints_2d.shape[0] == 0:
#                     print(f"No pose detected in frame {frame_count}")
#                 else:
#                     # Convert normalized keypoints to pixel coordinates
#                     for i in range(keypoints_2d.shape[0]):  # Iterate over detected poses
#                         if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
#                             kpt_2d = keypoints_2d[i]  # Get keypoints for person i
#                             kpt_3d = keypoints_3d[i]

#                             # Scale keypoints to original frame size
#                             kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
#                             kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

#                             frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

#                             # Store frame-wise keypoints
#                             for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
#                                 data_list.append([
#                                     frame_count, 
#                                     i, 
#                                     j, 
#                                     kpt_2d[j][0], 
#                                     kpt_2d[j][1], 
#                                     kpt_3d[j][0], 
#                                     kpt_3d[j][1], 
#                                     kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
#                                 ])
#                         else:
#                             print(f"Invalid keypoints for frame {frame_count}, person {i}")

#         out.write(frame)  # Write frame to output video
#         frame_count += 1

#     cap.release()
#     out.release()

#     # Convert collected data to a Pandas DataFrame
#     df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

#     # Save to Excel file
#     df.to_excel(output_excel, index=False)
#     print(f"Processed video saved as {output_video}")
#     print(f"Pose data saved as {output_excel}")
def process_video(video_path, model, skeleton_edges, output_video="output_video3d_withlabelschanged.mp4", frame_interval=10, output_excel="2dand3dposedatawithlabelschanged.xlsx"):
    """Process video frames with pose estimation, overlay skeletons and person labels, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_height, original_width = frame.shape[:2]
            frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
            image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

            # Run pose estimation
            pred = model.detect_poses(image_tensor, skeleton='smpl_24')

            if 'poses2d' in pred and 'poses3d' in pred:
                keypoints_2d = pred['poses2d'].numpy()
                keypoints_3d = pred['poses3d'].numpy()

                if keypoints_2d.shape[0] == 0:
                    print(f"No pose detected in frame {frame_count}")
                else:
                    # Iterate over detected poses (persons)
                    for i in range(keypoints_2d.shape[0]):
                        if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                            kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                            kpt_3d = keypoints_3d[i]

                            # Scale keypoints to original frame size
                            kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                            kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                            # Draw skeleton
                            frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                            # Add person label (e.g., "Person 1", "Person 2", etc.)
                            person_label = f"Person {i + 1}"  # Person ID starts from 1
                            label_position = (int(kpt_2d[0][0]), int(kpt_2d[0][1] - 20))  # Position above the head (e.g., pelvis or first keypoint)
                            cv2.putText(frame, person_label, label_position, 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

                            # Store frame-wise keypoints
                            for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                                data_list.append([
                                    frame_count, 
                                    i, 
                                    j, 
                                    kpt_2d[j][0], 
                                    kpt_2d[j][1], 
                                    kpt_3d[j][0], 
                                    kpt_3d[j][1], 
                                    kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                                ])
                        else:
                            print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out.write(frame)  # Write frame to output video
        frame_count += 1

    cap.release()
    out.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")

def calculate_gait_metrics_2d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 2D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_2d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    right_ankle_2d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_2D', 'Y_2D']].values

    # Detect heel strikes in 2D (lowest Y_2D position)
    left_strikes_2d = []
    right_strikes_2d = []
    for i in range(1, len(left_ankle_2d) - 1):
        if left_ankle_2d[i][2] < left_ankle_2d[i-1][2] and left_ankle_2d[i][2] < left_ankle_2d[i+1][2]:
            left_strikes_2d.append(left_ankle_2d[i])
        if right_ankle_2d[i][2] < right_ankle_2d[i-1][2] and right_ankle_2d[i][2] < right_ankle_2d[i+1][2]:
            right_strikes_2d.append(right_ankle_2d[i])

    left_strikes_2d = np.array(left_strikes_2d)
    right_strikes_2d = np.array(right_strikes_2d)

    # 1. Stride Time (2D)
    stride_time_left_2d = np.mean(np.diff(left_strikes_2d[:, 0]) / fps) if len(left_strikes_2d) > 1 else np.nan
    stride_time_right_2d = np.mean(np.diff(right_strikes_2d[:, 0]) / fps) if len(right_strikes_2d) > 1 else np.nan

    # 2. Stride Length (2D, in pixels)
    stride_length_left_2d = np.mean([distance.euclidean(left_strikes_2d[i][1:3], left_strikes_2d[i+1][1:3]) 
                                    for i in range(len(left_strikes_2d)-1)]) if len(left_strikes_2d) > 1 else np.nan
    stride_length_right_2d = np.mean([distance.euclidean(right_strikes_2d[i][1:3], right_strikes_2d[i+1][1:3]) 
                                     for i in range(len(right_strikes_2d)-1)]) if len(right_strikes_2d) > 1 else np.nan

    # 3. Cadence (2D)
    total_steps_2d = len(left_strikes_2d) + len(right_strikes_2d)
    total_time_2d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_2d = total_steps_2d / total_time_2d if total_time_2d > 0 else np.nan

    # 4. Gait Speed (2D, pixels per second)
    gait_speed_left_2d = stride_length_left_2d / stride_time_left_2d if stride_time_left_2d > 0 else np.nan
    gait_speed_right_2d = stride_length_right_2d / stride_time_right_2d if stride_time_right_2d > 0 else np.nan

    # 5. Knee and Hip Angles (2D)
    angles_2d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_2d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_2D', 'Y_2D']].values[0] if not row.empty else np.array([np.nan, np.nan])

        left_hip_2d = get_coords_2d('left_hip')
        left_knee_2d = get_coords_2d('left_knee')
        left_ankle_2d = get_coords_2d('left_ankle')
        right_hip_2d = get_coords_2d('right_hip')
        right_knee_2d = get_coords_2d('right_knee')
        right_ankle_2d = get_coords_2d('right_ankle')
        spine_2d = get_coords_2d('spine1')

        def angle_between_2d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_2d['left_knee'].append(angle_between_2d(left_hip_2d - left_knee_2d, left_ankle_2d - left_knee_2d))
        angles_2d['right_knee'].append(angle_between_2d(right_hip_2d - right_knee_2d, right_ankle_2d - right_knee_2d))
        angles_2d['left_hip'].append(angle_between_2d(spine_2d - left_hip_2d, left_knee_2d - left_hip_2d))
        angles_2d['right_hip'].append(angle_between_2d(spine_2d - right_hip_2d, right_knee_2d - right_hip_2d))

    avg_angles_2d = {k: np.nanmean(v) for k, v in angles_2d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_2d,
        'Stride Time Right (s)': stride_time_right_2d,
        'Stride Length Left (pixels)': stride_length_left_2d,
        'Stride Length Right (pixels)': stride_length_right_2d,
        'Cadence (steps/min)': cadence_2d,
        'Gait Speed Left (pixels/s)': gait_speed_left_2d,
        'Gait Speed Right (pixels/s)': gait_speed_right_2d,
        'Avg Left Knee Angle (deg)': avg_angles_2d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_2d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_2d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_2d['right_hip']
    }
def calculate_gait_metrics_3d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 3D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_3d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    right_ankle_3d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values

    # Detect heel strikes in 3D (lowest Z_3D position)
    left_strikes_3d = []
    right_strikes_3d = []
    for i in range(1, len(left_ankle_3d) - 1):
        if left_ankle_3d[i][3] < left_ankle_3d[i-1][3] and left_ankle_3d[i][3] < left_ankle_3d[i+1][3]:
            left_strikes_3d.append(left_ankle_3d[i])
        if right_ankle_3d[i][3] < right_ankle_3d[i-1][3] and right_ankle_3d[i][3] < right_ankle_3d[i+1][3]:
            right_strikes_3d.append(right_ankle_3d[i])

    left_strikes_3d = np.array(left_strikes_3d)
    right_strikes_3d = np.array(right_strikes_3d)

    # 1. Stride Time (3D)
    stride_time_left_3d = np.mean(np.diff(left_strikes_3d[:, 0]) / fps) if len(left_strikes_3d) > 1 else np.nan
    stride_time_right_3d = np.mean(np.diff(right_strikes_3d[:, 0]) / fps) if len(right_strikes_3d) > 1 else np.nan

    # 2. Stride Length (3D, assumed meters)
    stride_length_left_3d = np.mean([distance.euclidean(left_strikes_3d[i][1:4], left_strikes_3d[i+1][1:4]) 
                                    for i in range(len(left_strikes_3d)-1)]) if len(left_strikes_3d) > 1 else np.nan
    stride_length_right_3d = np.mean([distance.euclidean(right_strikes_3d[i][1:4], right_strikes_3d[i+1][1:4]) 
                                     for i in range(len(right_strikes_3d)-1)]) if len(right_strikes_3d) > 1 else np.nan

    # 3. Cadence (3D)
    total_steps_3d = len(left_strikes_3d) + len(right_strikes_3d)
    total_time_3d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_3d = total_steps_3d / total_time_3d if total_time_3d > 0 else np.nan

    # 4. Gait Speed (3D, assumed meters per second)
    gait_speed_left_3d = stride_length_left_3d / stride_time_left_3d if stride_time_left_3d > 0 else np.nan
    gait_speed_right_3d = stride_length_right_3d / stride_time_right_3d if stride_time_right_3d > 0 else np.nan

    # 5. Knee and Hip Angles (3D)
    angles_3d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_3d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_3D', 'Y_3D', 'Z_3D']].values[0] if not row.empty else np.array([np.nan, np.nan, np.nan])

        left_hip_3d = get_coords_3d('left_hip')
        left_knee_3d = get_coords_3d('left_knee')
        left_ankle_3d = get_coords_3d('left_ankle')
        right_hip_3d = get_coords_3d('right_hip')
        right_knee_3d = get_coords_3d('right_knee')
        right_ankle_3d = get_coords_3d('right_ankle')
        spine_3d = get_coords_3d('spine1')

        def angle_between_3d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_3d['left_knee'].append(angle_between_3d(left_hip_3d - left_knee_3d, left_ankle_3d - left_knee_3d))
        angles_3d['right_knee'].append(angle_between_3d(right_hip_3d - right_knee_3d, right_ankle_3d - right_knee_3d))
        angles_3d['left_hip'].append(angle_between_3d(spine_3d - left_hip_3d, left_knee_3d - left_hip_3d))
        angles_3d['right_hip'].append(angle_between_3d(spine_3d - right_hip_3d, right_knee_3d - right_hip_3d))

    avg_angles_3d = {k: np.nanmean(v) for k, v in angles_3d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_3d,
        'Stride Time Right (s)': stride_time_right_3d,
        'Stride Length Left (m)': stride_length_left_3d,
        'Stride Length Right (m)': stride_length_right_3d,
        'Cadence (steps/min)': cadence_3d,
        'Gait Speed Left (m/s)': gait_speed_left_3d,
        'Gait Speed Right (m/s)': gait_speed_right_3d,
        'Avg Left Knee Angle (deg)': avg_angles_3d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_3d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_3d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_3d['right_hip']
    }
def main():
    # Paths
    zip_path = "C:\\Users\\akhileshsing2024\\Downloads\\metrabs_mob3l_y4t.zip"
    video_path = "C:\\Users\\akhileshsing2024\\Downloads\\video.mp4"
    output_excel = "2dand3dposedatawithlabelschanged.xlsx"

    # Extract and load the model
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    #model.summary()
    print(dir(model))
    print("Model loaded successfully!")

    # Load skeleton information
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()
    

    # Process video
    process_video(video_path, model, joint_edges, output_excel=output_excel)

    # Load the generated Excel file
    df = pd.read_excel(output_excel)
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)

    # Determine the number of unique persons detected
    unique_persons = df['Person'].unique()
    print(f"Detected {len(unique_persons)} persons in the video.")

    # Calculate gait metrics for each detected person
    metrics_2d_all = []
    metrics_3d_all = []
    for person_id in unique_persons:
        print(f"\nProcessing Person {person_id}...")
        
        # 2D Metrics
        metrics_2d = calculate_gait_metrics_2d(df, person_id, fps, joint_edges, joint_names)
        if metrics_2d:
            metrics_2d_all.append(metrics_2d)
            print(f"2D Metrics for Person {person_id}:")
            for key, value in metrics_2d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

        # 3D Metrics
        metrics_3d = calculate_gait_metrics_3d(df, person_id, fps, joint_edges, joint_names)
        if metrics_3d:
            metrics_3d_all.append(metrics_3d)
            print(f"3D Metrics for Person {person_id}:")
            for key, value in metrics_3d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    # Save to Excel with separate sheets for 2D and 3D
    metrics_df_2d = pd.DataFrame(metrics_2d_all)
    metrics_df_3d = pd.DataFrame(metrics_3d_all)
    with pd.ExcelWriter("gait_metrics_per_person.xlsx") as writer:
        metrics_df_2d.to_excel(writer, sheet_name='2D Metrics', index=False)
        metrics_df_3d.to_excel(writer, sheet_name='3D Metrics', index=False)
    print("\nGait metrics for all detected persons saved to 'gait_metrics_per_person.xlsx'")

if __name__ == '__main__':
    main()

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_add_trackable_child', '_add_variable_with_custom_getter', '_checkpoint_dependencies', '_deferred_dependencies', '_delete_tracking', '_deserialization_dependencies', '_deserialize_from_proto', '_export_to_saved_model_graph', '_gather_saveables_for_checkpoint', '_handle_deferred_dependencies', '_lookup_dependency', '_maybe_initialize_trackable', '_name_based_attribute_restore', '_name_based_restores', '_no_dependency', '_object_identifier', '_preload_simple_restoration', '_restore_from_tensors', '_self_name_based_restores', '_self_saveable_object_factories', '_self_setattr_tracking', '_self_unconditional_checkpoint_dependencies', '_self_unconditio

In [11]:
def main():
    # Paths
    zip_path = "C:\\Users\\akhileshsing2024\\Downloads\\metrabs_mob3l_y4t.zip"
    video_path = "C:\\Users\\akhileshsing2024\\Downloads\\video.mp4"
    output_excel = "2dand3dposedatawithlabelschanged.xlsx"

    # Extract and load the model
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")

    # Load skeleton information
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()

    # Process video (this generates the initial Excel file with 2D in pixels, 3D in mm)
    process_video(video_path, model, joint_edges, output_excel=output_excel)

    # Load the generated Excel file
    df = pd.read_excel(output_excel)

    # Step 1: Compute the intrinsic matrix for the resized 640x480 image
    width, height = 640, 480  # Resized image dimensions
    fov_degrees = 55  # Default FOV from MeTRabs
    f_x = (width / 2) / np.tan(np.radians(fov_degrees / 2))
    f_y = f_x * (height / width)
    c_x = width / 2
    c_y = height / 2
    intrinsic_matrix = np.array([
        [f_x, 0, c_x],
        [0, f_y, c_y],
        [0, 0, 1]
    ])
    print(f"Computed intrinsic matrix for 640x480 image:\n{intrinsic_matrix}")

    # Adjust intrinsic matrix for original frame size
    cap = cv2.VideoCapture(video_path)
    original_width = int(cap.get(3))
    original_height = int(cap.get(4))
    cap.release()
    scale_x = original_width / 640
    scale_y = original_height / 480
    intrinsic_matrix[0, 0] *= scale_x  # Adjust f_x
    intrinsic_matrix[1, 1] *= scale_y  # Adjust f_y
    intrinsic_matrix[0, 2] *= scale_x  # Adjust c_x
    intrinsic_matrix[1, 2] *= scale_y  # Adjust c_y
    print(f"Adjusted intrinsic matrix for original {original_width}x{original_height} resolution:\n{intrinsic_matrix}")

    # Step 2: Compute pixel-to-mm scaling factors using depth
    unique_persons = df['Person'].unique()
    scale_factors_x = []
    scale_factors_y = []

    for person_id in unique_persons:
        df_person = df[df['Person'] == person_id]
        for frame in df_person['Frame'].unique():
            frame_data = df_person[df_person['Frame'] == frame]
            for joint in frame_data['Joint'].unique():
                joint_data = frame_data[frame_data['Joint'] == joint]
                if not joint_data.empty:
                    u = joint_data['X_2D'].values[0]
                    v = joint_data['Y_2D'].values[0]
                    X = joint_data['X_3D'].values[0]
                    Y = joint_data['Y_3D'].values[0]
                    Z = joint_data['Z_3D'].values[0]

                    if Z != 0:  # Avoid division by zero
                        # Compute scaling factors using the pinhole model
                        scale_x = X / (u - intrinsic_matrix[0, 2])
                        scale_y = Y / (v - intrinsic_matrix[1, 2])

                        # Alternative: Use Z directly
                        # scale_x = Z / intrinsic_matrix[0, 0]
                        # scale_y = Z / intrinsic_matrix[1, 1]

                        if np.isfinite(scale_x) and np.isfinite(scale_y):
                            scale_factors_x.append(scale_x)
                            scale_factors_y.append(scale_y)

    # Compute average scaling factors
    if scale_factors_x and scale_factors_y:
        scale_x = np.mean(scale_factors_x)
        scale_y = np.mean(scale_factors_y)
        print(f"Computed pixel-to-mm scaling factors: scale_x = {scale_x:.4f} mm/pixel, scale_y = {scale_y:.4f} mm/pixel")
    else:
        # Fallback: Use a reasonable default based on typical depth
        scale_x = scale_y = 1.0  # Placeholder; adjust based on typical Z values if needed
        print("Could not compute scaling factors. Using fallback: 1.0 mm/pixel")

    # Step 3: Convert 2D coordinates to millimeters
    df['X_2D'] = (df['X_2D'] - intrinsic_matrix[0, 2]) * scale_x
    df['Y_2D'] = (df['Y_2D'] - intrinsic_matrix[1, 2]) * scale_y

    # Step 4: Save the updated Excel file with 2D and 3D coordinates in millimeters
    updated_excel = "2dand3dposedatawithlabelschanged_converted.xlsx"
    df.to_excel(updated_excel, index=False)
    print(f"Updated pose data with 2D coordinates in millimeters saved as {updated_excel}")

    # Step 5: Calculate gait metrics
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
    print(f"Detected {len(unique_persons)} persons in the video.")

    metrics_2d_all = []
    metrics_3d_all = []
    for person_id in unique_persons:
        print(f"\nProcessing Person {person_id}...")
        
        # 2D Metrics (now in millimeters)
        metrics_2d = calculate_gait_metrics_2d(df, person_id, fps, joint_edges, joint_names)
        if metrics_2d:
            metrics_2d_all.append(metrics_2d)
            print(f"2D Metrics for Person {person_id}:")
            for key, value in metrics_2d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

        # 3D Metrics (already in millimeters)
        metrics_3d = calculate_gait_metrics_3d(df, person_id, fps, joint_edges, joint_names)
        if metrics_3d:
            metrics_3d_all.append(metrics_3d)
            print(f"3D Metrics for Person {person_id}:")
            for key, value in metrics_3d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    # Save to Excel with separate sheets for 2D and 3D
    metrics_df_2d = pd.DataFrame(metrics_2d_all)
    metrics_df_3d = pd.DataFrame(metrics_3d_all)
    with pd.ExcelWriter("gait_metrics_per_person.xlsx") as writer:
        metrics_df_2d.to_excel(writer, sheet_name='2D Metrics', index=False)
        metrics_df_3d.to_excel(writer, sheet_name='3D Metrics', index=False)
    print("\nGait metrics for all detected persons saved to 'gait_metrics_per_person.xlsx'")

In [ ]:
import sys
import urllib.request
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_io as tfio
import cv2
import os
import zipfile
import numpy as np
import pandas as pd
from scipy.spatial import distance

def extract_model(zip_path, extract_to="models"):
    """Extracts a zip file and returns the extracted directory path."""
    extract_dir = os.path.join(os.path.dirname(zip_path), extract_to)

    if not os.path.exists(extract_dir):
        os.makedirs(extract_dir)

    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(extract_dir)

    model_dir = os.path.join(extract_dir, os.path.splitext(os.path.basename(zip_path))[0])
    return model_dir

def draw_skeleton(frame, keypoints, skeleton_edges):
    """Draws skeleton on the frame using detected keypoints."""
    for (i, j) in skeleton_edges:
        pt1, pt2 = keypoints[i], keypoints[j]

        # Check if confidence is available (shape [N, 24, 3] expected)
        if keypoints.shape[-1] == 3:  
            if (pt1[2] > 0.5) and (pt2[2] > 0.5):  # Confidence threshold
                cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)  # Blue lines
        else:  
            # Draw without confidence check
            cv2.line(frame, (int(pt1[0]), int(pt1[1])), (int(pt2[0]), int(pt2[1])), (255, 0, 0), 2)

    return frame

# def process_video(video_path, model, skeleton_edges, output_video="output_video3d-36.mp4", frame_interval=10, output_excel="2dand3dposedata.xlsx"):
#     """Process video frames with pose estimation, overlay results, and save coordinates to an Excel file."""
    
#     cap = cv2.VideoCapture(video_path)
#     frame_width = int(cap.get(3))
#     frame_height = int(cap.get(4))
#     fps = int(cap.get(cv2.CAP_PROP_FPS))

#     fourcc = cv2.VideoWriter_fourcc(*'mp4v')
#     out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

#     frame_count = 0
#     data_list = []  # Store pose data

#     while cap.isOpened():
#         ret, frame = cap.read()
#         if not ret:
#             break

#         if frame_count % frame_interval == 0:
#             frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             original_height, original_width = frame.shape[:2]
#             frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
#             image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

#             # Run pose estimation
#             pred = model.detect_poses(image_tensor, skeleton='smpl_24')

#             if 'poses2d' in pred and 'poses3d' in pred:
#                 keypoints_2d = pred['poses2d'].numpy()
#                 keypoints_3d = pred['poses3d'].numpy()

#                 if keypoints_2d.shape[0] == 0:
#                     print(f"No pose detected in frame {frame_count}")
#                 else:
#                     # Convert normalized keypoints to pixel coordinates
#                     for i in range(keypoints_2d.shape[0]):  # Iterate over detected poses
#                         if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
#                             kpt_2d = keypoints_2d[i]  # Get keypoints for person i
#                             kpt_3d = keypoints_3d[i]

#                             # Scale keypoints to original frame size
#                             kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
#                             kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

#                             frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

#                             # Store frame-wise keypoints
#                             for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
#                                 data_list.append([
#                                     frame_count, 
#                                     i, 
#                                     j, 
#                                     kpt_2d[j][0], 
#                                     kpt_2d[j][1], 
#                                     kpt_3d[j][0], 
#                                     kpt_3d[j][1], 
#                                     kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
#                                 ])
#                         else:
#                             print(f"Invalid keypoints for frame {frame_count}, person {i}")

#         out.write(frame)  # Write frame to output video
#         frame_count += 1

#     cap.release()
#     out.release()

#     # Convert collected data to a Pandas DataFrame
#     df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

#     # Save to Excel file
#     df.to_excel(output_excel, index=False)
#     print(f"Processed video saved as {output_video}")
#     print(f"Pose data saved as {output_excel}")
def process_video(video_path, model, skeleton_edges, output_video="output_video3d_withlabelschanged.mp4", frame_interval=10, output_excel="2dand3dposedatawithlabelschanged.xlsx"):
    """Process video frames with pose estimation, overlay skeletons and person labels, and save coordinates to an Excel file."""
    
    cap = cv2.VideoCapture(video_path)
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (frame_width, frame_height))

    frame_count = 0
    data_list = []  # Store pose data

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            original_height, original_width = frame.shape[:2]
            frame_rgb_resized = cv2.resize(frame_rgb, (640, 480))
            image_tensor = tf.convert_to_tensor(frame_rgb_resized, dtype=tf.uint8)

            # Run pose estimation
            pred = model.detect_poses(image_tensor, skeleton='smpl_24')

            if 'poses2d' in pred and 'poses3d' in pred:
                keypoints_2d = pred['poses2d'].numpy()
                keypoints_3d = pred['poses3d'].numpy()

                if keypoints_2d.shape[0] == 0:
                    print(f"No pose detected in frame {frame_count}")
                else:
                    # Iterate over detected poses (persons)
                    for i in range(keypoints_2d.shape[0]):
                        if keypoints_2d[i].shape[0] > 0 and keypoints_3d[i].shape[0] > 0:  # Check for valid keypoints
                            kpt_2d = keypoints_2d[i]  # Get keypoints for person i
                            kpt_3d = keypoints_3d[i]

                            # Scale keypoints to original frame size
                            kpt_2d[:, 0] *= original_width / 640  # Scale x-coordinates
                            kpt_2d[:, 1] *= original_height / 480  # Scale y-coordinates

                            # Draw skeleton
                            frame = draw_skeleton(frame, kpt_2d, skeleton_edges)

                            # Add person label (e.g., "Person 1", "Person 2", etc.)
                            person_label = f"Person {i + 1}"  # Person ID starts from 1
                            label_position = (int(kpt_2d[0][0]), int(kpt_2d[0][1] - 20))  # Position above the head (e.g., pelvis or first keypoint)
                            cv2.putText(frame, person_label, label_position, 
                                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2, cv2.LINE_AA)

                            # Store frame-wise keypoints
                            for j in range(min(kpt_2d.shape[0], kpt_3d.shape[0])):  # Ensure we only access valid joints
                                data_list.append([
                                    frame_count, 
                                    i, 
                                    j, 
                                    kpt_2d[j][0], 
                                    kpt_2d[j][1], 
                                    kpt_3d[j][0], 
                                    kpt_3d[j][1], 
                                    kpt_3d[j][2],  # Include Z coordinate for 3D keypoints
                                ])
                        else:
                            print(f"Invalid keypoints for frame {frame_count}, person {i}")

        out.write(frame)  # Write frame to output video
        frame_count += 1

    cap.release()
    out.release()

    # Convert collected data to a Pandas DataFrame
    df = pd.DataFrame(data_list, columns=['Frame', 'Person', 'Joint', 'X_2D', 'Y_2D', 'X_3D', 'Y_3D', 'Z_3D'])

    # Save to Excel file
    df.to_excel(output_excel, index=False)
    print(f"Processed video saved as {output_video}")
    print(f"Pose data saved as {output_excel}")

def calculate_gait_metrics_2d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 2D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_2d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_2D', 'Y_2D']].values
    right_ankle_2d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_2D', 'Y_2D']].values

    # Detect heel strikes in 2D (lowest Y_2D position)
    left_strikes_2d = []
    right_strikes_2d = []
    for i in range(1, len(left_ankle_2d) - 1):
        if left_ankle_2d[i][2] < left_ankle_2d[i-1][2] and left_ankle_2d[i][2] < left_ankle_2d[i+1][2]:
            left_strikes_2d.append(left_ankle_2d[i])
        if right_ankle_2d[i][2] < right_ankle_2d[i-1][2] and right_ankle_2d[i][2] < right_ankle_2d[i+1][2]:
            right_strikes_2d.append(right_ankle_2d[i])

    left_strikes_2d = np.array(left_strikes_2d)
    right_strikes_2d = np.array(right_strikes_2d)

    # 1. Stride Time (2D)
    stride_time_left_2d = np.mean(np.diff(left_strikes_2d[:, 0]) / fps) if len(left_strikes_2d) > 1 else np.nan
    stride_time_right_2d = np.mean(np.diff(right_strikes_2d[:, 0]) / fps) if len(right_strikes_2d) > 1 else np.nan

    # 2. Stride Length (2D, in pixels)
    stride_length_left_2d = np.mean([distance.euclidean(left_strikes_2d[i][1:3], left_strikes_2d[i+1][1:3]) 
                                    for i in range(len(left_strikes_2d)-1)]) if len(left_strikes_2d) > 1 else np.nan
    stride_length_right_2d = np.mean([distance.euclidean(right_strikes_2d[i][1:3], right_strikes_2d[i+1][1:3]) 
                                     for i in range(len(right_strikes_2d)-1)]) if len(right_strikes_2d) > 1 else np.nan

    # 3. Cadence (2D)
    total_steps_2d = len(left_strikes_2d) + len(right_strikes_2d)
    total_time_2d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_2d = total_steps_2d / total_time_2d if total_time_2d > 0 else np.nan

    # 4. Gait Speed (2D, pixels per second)
    gait_speed_left_2d = stride_length_left_2d / stride_time_left_2d if stride_time_left_2d > 0 else np.nan
    gait_speed_right_2d = stride_length_right_2d / stride_time_right_2d if stride_time_right_2d > 0 else np.nan

    # 5. Knee and Hip Angles (2D)
    angles_2d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_2d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_2D', 'Y_2D']].values[0] if not row.empty else np.array([np.nan, np.nan])

        left_hip_2d = get_coords_2d('left_hip')
        left_knee_2d = get_coords_2d('left_knee')
        left_ankle_2d = get_coords_2d('left_ankle')
        right_hip_2d = get_coords_2d('right_hip')
        right_knee_2d = get_coords_2d('right_knee')
        right_ankle_2d = get_coords_2d('right_ankle')
        spine_2d = get_coords_2d('spine1')

        def angle_between_2d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_2d['left_knee'].append(angle_between_2d(left_hip_2d - left_knee_2d, left_ankle_2d - left_knee_2d))
        angles_2d['right_knee'].append(angle_between_2d(right_hip_2d - right_knee_2d, right_ankle_2d - right_knee_2d))
        angles_2d['left_hip'].append(angle_between_2d(spine_2d - left_hip_2d, left_knee_2d - left_hip_2d))
        angles_2d['right_hip'].append(angle_between_2d(spine_2d - right_hip_2d, right_knee_2d - right_hip_2d))

    avg_angles_2d = {k: np.nanmean(v) for k, v in angles_2d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_2d,
        'Stride Time Right (s)': stride_time_right_2d,
        'Stride Length Left (mm)': stride_length_left_2d,
        'Stride Length Right (mm)': stride_length_right_2d,
        'Cadence (steps/min)': cadence_2d,
        'Gait Speed Left (mm/s)': gait_speed_left_2d,
        'Gait Speed Right (mm/s)': gait_speed_right_2d,
        'Avg Left Knee Angle (deg)': avg_angles_2d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_2d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_2d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_2d['right_hip']
    }
def calculate_gait_metrics_3d(df, person_id, fps, skeleton_edges, joint_names):
    """Calculate 3D gait metrics for a specific person."""
    joint_map = {
        'pelvis': 0, 'left_hip': 1, 'right_hip': 2, 'spine1': 3,
        'left_knee': 4, 'right_knee': 5, 'spine2': 6,
        'left_ankle': 7, 'right_ankle': 8
    }

    df_person = df[df['Person'] == person_id]
    if df_person.empty:
        return None  # No data for this person

    left_ankle_3d = df_person[df_person['Joint'] == joint_map['left_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values
    right_ankle_3d = df_person[df_person['Joint'] == joint_map['right_ankle']][['Frame', 'X_3D', 'Y_3D', 'Z_3D']].values

    # Detect heel strikes in 3D (lowest Z_3D position)
    left_strikes_3d = []
    right_strikes_3d = []
    for i in range(1, len(left_ankle_3d) - 1):
        if left_ankle_3d[i][3] < left_ankle_3d[i-1][3] and left_ankle_3d[i][3] < left_ankle_3d[i+1][3]:
            left_strikes_3d.append(left_ankle_3d[i])
        if right_ankle_3d[i][3] < right_ankle_3d[i-1][3] and right_ankle_3d[i][3] < right_ankle_3d[i+1][3]:
            right_strikes_3d.append(right_ankle_3d[i])

    left_strikes_3d = np.array(left_strikes_3d)
    right_strikes_3d = np.array(right_strikes_3d)

    # 1. Stride Time (3D)
    stride_time_left_3d = np.mean(np.diff(left_strikes_3d[:, 0]) / fps) if len(left_strikes_3d) > 1 else np.nan
    stride_time_right_3d = np.mean(np.diff(right_strikes_3d[:, 0]) / fps) if len(right_strikes_3d) > 1 else np.nan

    # 2. Stride Length (3D, assumed meters)
    stride_length_left_3d = np.mean([distance.euclidean(left_strikes_3d[i][1:4], left_strikes_3d[i+1][1:4]) 
                                    for i in range(len(left_strikes_3d)-1)]) if len(left_strikes_3d) > 1 else np.nan
    stride_length_right_3d = np.mean([distance.euclidean(right_strikes_3d[i][1:4], right_strikes_3d[i+1][1:4]) 
                                     for i in range(len(right_strikes_3d)-1)]) if len(right_strikes_3d) > 1 else np.nan

    # 3. Cadence (3D)
    total_steps_3d = len(left_strikes_3d) + len(right_strikes_3d)
    total_time_3d = (df_person['Frame'].max() - df_person['Frame'].min()) / fps / 60 if not df_person['Frame'].empty else 0
    cadence_3d = total_steps_3d / total_time_3d if total_time_3d > 0 else np.nan

    # 4. Gait Speed (3D, assumed meters per second)
    gait_speed_left_3d = stride_length_left_3d / stride_time_left_3d if stride_time_left_3d > 0 else np.nan
    gait_speed_right_3d = stride_length_right_3d / stride_time_right_3d if stride_time_right_3d > 0 else np.nan

    # 5. Knee and Hip Angles (3D)
    angles_3d = {'left_knee': [], 'right_knee': [], 'left_hip': [], 'right_hip': []}
    for frame in df_person['Frame'].unique():
        frame_data = df_person[df_person['Frame'] == frame]
        
        def get_coords_3d(joint):
            row = frame_data[frame_data['Joint'] == joint_map[joint]]
            return row[['X_3D', 'Y_3D', 'Z_3D']].values[0] if not row.empty else np.array([np.nan, np.nan, np.nan])

        left_hip_3d = get_coords_3d('left_hip')
        left_knee_3d = get_coords_3d('left_knee')
        left_ankle_3d = get_coords_3d('left_ankle')
        right_hip_3d = get_coords_3d('right_hip')
        right_knee_3d = get_coords_3d('right_knee')
        right_ankle_3d = get_coords_3d('right_ankle')
        spine_3d = get_coords_3d('spine1')

        def angle_between_3d(v1, v2):
            if np.any(np.isnan(v1)) or np.any(np.isnan(v2)):
                return np.nan
            cos_theta = np.dot(v1, v2) / (np.linalg.norm(v1) * np.linalg.norm(v2))
            return np.degrees(np.arccos(np.clip(cos_theta, -1.0, 1.0)))

        angles_3d['left_knee'].append(angle_between_3d(left_hip_3d - left_knee_3d, left_ankle_3d - left_knee_3d))
        angles_3d['right_knee'].append(angle_between_3d(right_hip_3d - right_knee_3d, right_ankle_3d - right_knee_3d))
        angles_3d['left_hip'].append(angle_between_3d(spine_3d - left_hip_3d, left_knee_3d - left_hip_3d))
        angles_3d['right_hip'].append(angle_between_3d(spine_3d - right_hip_3d, right_knee_3d - right_hip_3d))

    avg_angles_3d = {k: np.nanmean(v) for k, v in angles_3d.items()}

    return {
        'Person': person_id,
        'Stride Time Left (s)': stride_time_left_3d,
        'Stride Time Right (s)': stride_time_right_3d,
        'Stride Length Left (mm)': stride_length_left_3d,
        'Stride Length Right (mm)': stride_length_right_3d,
        'Cadence (steps/min)': cadence_3d,
        'Gait Speed Left (mm/s)': gait_speed_left_3d,
        'Gait Speed Right (mm/s)': gait_speed_right_3d,
        'Avg Left Knee Angle (deg)': avg_angles_3d['left_knee'],
        'Avg Right Knee Angle (deg)': avg_angles_3d['right_knee'],
        'Avg Left Hip Angle (deg)': avg_angles_3d['left_hip'],
        'Avg Right Hip Angle (deg)': avg_angles_3d['right_hip']
    }

def main():
    # Paths
    zip_path = "C:\\Users\\akhileshsing2024\\Downloads\\metrabs_mob3l_y4t.zip"
    video_path = "C:\\Users\\akhileshsing2024\\Downloads\\video.mp4"
    output_excel = "2dand3dposedatawithlabelschanged.xlsx"

    # Extract and load the model
    model_path = extract_model(zip_path)
    model = tf.saved_model.load(model_path)
    print("Model loaded successfully!")

    # Load skeleton information
    skeleton = 'smpl_24'
    joint_names = model.per_skeleton_joint_names[skeleton].numpy().astype(str)
    joint_edges = model.per_skeleton_joint_edges[skeleton].numpy()

    # Process video (this generates the initial Excel file with 2D in pixels, 3D in mm)
    process_video(video_path, model, joint_edges, output_excel=output_excel)

    # Load the generated Excel file
    df = pd.read_excel(output_excel)

    # Step 1: Compute pixel-to-mm conversion factor using hip-to-ankle distance
    unique_persons = df['Person'].unique()
    pixel_to_mm_factors = []

    for person_id in unique_persons:
        df_person = df[df['Person'] == person_id]
        
        # Extract hip and ankle coordinates for all frames
        left_hip_2d = df_person[df_person['Joint'] == 1][['Frame', 'X_2D', 'Y_2D']]
        left_ankle_2d = df_person[df_person['Joint'] == 7][['Frame', 'X_2D', 'Y_2D']]
        left_hip_3d = df_person[df_person['Joint'] == 1][['Frame', 'X_3D', 'Y_3D', 'Z_3D']]
        left_ankle_3d = df_person[df_person['Joint'] == 7][['Frame', 'X_3D', 'Y_3D', 'Z_3D']]

        # Merge on Frame to get corresponding hip and ankle coordinates
        hip_ankle_2d = left_hip_2d.merge(left_ankle_2d, on='Frame', suffixes=('_hip', '_ankle'))
        hip_ankle_3d = left_hip_3d.merge(left_ankle_3d, on='Frame', suffixes=('_hip', '_ankle'))

        # Compute distances for each frame
        distances_2d = []
        distances_3d = []
        for idx in range(len(hip_ankle_2d)):
            # 2D distance (pixels)
            hip_2d = hip_ankle_2d.iloc[idx][['X_2D_hip', 'Y_2D_hip']].values
            ankle_2d = hip_ankle_2d.iloc[idx][['X_2D_ankle', 'Y_2D_ankle']].values
            dist_2d = distance.euclidean(hip_2d, ankle_2d)

            # 3D distance (mm)
            hip_3d = hip_ankle_3d.iloc[idx][['X_3D_hip', 'Y_3D_hip', 'Z_3D_hip']].values
            ankle_3d = hip_ankle_3d.iloc[idx][['X_3D_ankle', 'Y_3D_ankle', 'Z_3D_ankle']].values
            dist_3d = distance.euclidean(hip_3d, ankle_3d)

            if dist_2d > 0:  # Avoid division by zero
                pixel_to_mm = dist_3d / dist_2d
                pixel_to_mm_factors.append(pixel_to_mm)
                distances_2d.append(dist_2d)
                distances_3d.append(dist_3d)

    # Compute the average pixel-to-mm factor
    if pixel_to_mm_factors:
        pixel_to_mm = np.mean(pixel_to_mm_factors)
        print(f"Computed pixel-to-mm conversion factor: {pixel_to_mm:.4f} mm/pixel")
        print(f"Average 3D hip-to-ankle distance: {np.mean(distances_3d):.2f} mm")
        print(f"Average 2D hip-to-ankle distance: {np.mean(distances_2d):.2f} pixels")
    else:
        # Fallback to anthropometric average if no valid distances are computed
        pixel_to_mm = 900 / 500  # Assume 900 mm leg length, 500 pixels in 2D (rough estimate)
        print(f"No valid distances computed. Using fallback pixel-to-mm factor: {pixel_to_mm:.4f} mm/pixel")

    # Step 2: Convert 2D coordinates to millimeters
    df['X_2D'] = df['X_2D'] * pixel_to_mm
    df['Y_2D'] = df['Y_2D'] * pixel_to_mm

    # Step 3: Save the updated Excel file with 2D and 3D coordinates in millimeters
    updated_excel = "2dand3dposedatawithlabelschanged_converted.xlsx"
    df.to_excel(updated_excel, index=False)
    print(f"Updated pose data with 2D coordinates in millimeters saved as {updated_excel}")

    # Step 4: Calculate gait metrics
    fps = cv2.VideoCapture(video_path).get(cv2.CAP_PROP_FPS)
    print(f"Detected {len(unique_persons)} persons in the video.")

    metrics_2d_all = []
    metrics_3d_all = []
    for person_id in unique_persons:
        print(f"\nProcessing Person {person_id}...")
        
        # 2D Metrics (now in millimeters)
        metrics_2d = calculate_gait_metrics_2d(df, person_id, fps, joint_edges, joint_names)
        if metrics_2d:
            metrics_2d_all.append(metrics_2d)
            print(f"2D Metrics for Person {person_id}:")
            for key, value in metrics_2d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

        # 3D Metrics (already in millimeters)
        metrics_3d = calculate_gait_metrics_3d(df, person_id, fps, joint_edges, joint_names)
        if metrics_3d:
            metrics_3d_all.append(metrics_3d)
            print(f"3D Metrics for Person {person_id}:")
            for key, value in metrics_3d.items():
                if key != 'Person':
                    print(f"{key}: {value:.2f}" if not np.isnan(value) else f"{key}: N/A")

    # Save to Excel with separate sheets for 2D and 3D
    metrics_df_2d = pd.DataFrame(metrics_2d_all)
    metrics_df_3d = pd.DataFrame(metrics_3d_all)
    with pd.ExcelWriter("gait_metrics_per_person.xlsx") as writer:
        metrics_df_2d.to_excel(writer, sheet_name='2D Metrics', index=False)
        metrics_df_3d.to_excel(writer, sheet_name='3D Metrics', index=False)
    print("\nGait metrics for all detected persons saved to 'gait_metrics_per_person.xlsx'")
if __name__ == '__main__':
    main()

Model loaded successfully!
Processed video saved as output_video3d_withlabelschanged.mp4
Pose data saved as 2dand3dposedatawithlabelschanged.xlsx
Computed pixel-to-mm conversion factor: 3.0609 mm/pixel
Average 3D hip-to-ankle distance: 537.42 mm
Average 2D hip-to-ankle distance: 107.36 pixels
Updated pose data with 2D coordinates in millimeters saved as 2dand3dposedatawithlabelschanged_converted.xlsx
Detected 8 persons in the video.

Processing Person 0...
2D Metrics for Person 0:
Stride Time Left (s): 1.25
Stride Time Right (s): 1.25
Stride Length Left (mm): 1467.24
Stride Length Right (mm): 1075.66
Cadence (steps/min): 95.90
Gait Speed Left (mm/s): 1172.62
Gait Speed Right (mm/s): 859.67
Avg Left Knee Angle (deg): 172.90
Avg Right Knee Angle (deg): 160.95
Avg Left Hip Angle (deg): 170.35
Avg Right Hip Angle (deg): 160.23
3D Metrics for Person 0:
Stride Time Left (s): 0.97
Stride Time Right (s): 1.08
Stride Length Left (mm): 1597.56
Stride Length Right (mm): 1089.24
Cadence (steps/min